In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
import zipfile,os
local_zip = '/content/holoen.zip' #change the zip filename
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
train_dir = '/content/holoen/train' #change the middle one to the zip parent folder
validation_dir = '/content/holoen/test' #change the middle one to the zip parent folder

In [ ]:
train_datagen = ImageDataGenerator(
                    validation_split=0.2,
                    rescale=1./255)
 
test_datagen = ImageDataGenerator(
                    validation_split=0.2,
                    rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode='categorical',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    shuffle=True)

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                         class_mode='categorical',
                                                         target_size=(150, 150),
                                                         batch_size=32,
                                                         shuffle=True)

Found 1000 images belonging to 5 classes.
Found 200 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax') #change the number to the number of characters in your zip file
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['categorical_accuracy'])

In [ ]:
filepath = "/content/model-hololive-en.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=12,
      epochs=6,
      validation_data=validation_generator,
      validation_steps=12,
      verbose=1,
      callbacks=desired_callbacks)

Epoch 1/32
12/12 [==============================] - 20s 2s/step - loss: 1.6373 - categorical_accuracy: 0.1722 - val_loss: 1.6067 - val_categorical_accuracy: 0.2000

Epoch 00001: loss improved from inf to 1.63734, saving model to /content/model-hololive-en.hdf5
Epoch 2/32
12/12 [==============================] - 17s 1s/step - loss: 1.5857 - categorical_accuracy: 0.2396

Epoch 00002: loss improved from 1.63734 to 1.58567, saving model to /content/model-hololive-en.hdf5
Epoch 3/32
12/12 [==============================] - 16s 1s/step - loss: 1.5488 - categorical_accuracy: 0.3306

Epoch 00003: loss improved from 1.58567 to 1.54882, saving model to /content/model-hololive-en.hdf5
Epoch 4/32
12/12 [==============================] - 18s 1s/step - loss: 1.4327 - categorical_accuracy: 0.4167

Epoch 00004: loss improved from 1.54882 to 1.43267, saving model to /content/model-hololive-en.hdf5
Epoch 5/32
12/12 [==============================] - 17s 1s/step - loss: 1.3144 - categorical_accuracy: 0.4